In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima.model import ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
import pmdarima as pm
from pmdarima import model_selection
import pickle as pkl

Let's start by reading in the data

In [ ]:
#read in set
df = pd.read_csv()

Now change the index to a timestamp

In [ ]:
#change the time to timestamp and reindex
df[time_col] = pd.to_datetime(df[time_col])
df.set_index(df[time_col],inplace=True)

Make the first raw time series plot

In [ ]:
#plot the ra wseries
df[metric].plot()

Try an additive decompose

In [ ]:
#decompose additive
result = seasonal_decompose(df[metric], model='additive')
result.plot()
plt.show()

Try a multiplicative decompose

In [ ]:
#decompose multiplicative
result = seasonal_decompose(df['#Passengers'], model='multiplicative')
result.plot()
plt.show()

Try testing for stationarity with the ADFuller Test

In [ ]:
#perform adfuller test
result = adfuller(df[metric])
print(result)

Try seeing what differentiating does

In [ ]:
#differentiate the series and plot
df['diff_series'] = df[metric].diff()
df['diff_series'].plot()

Let's take a look at the seasonality

In [ ]:
#find the trend year by year
df['month_num'] = [i.month for i in df['Month']]
yearly_trend = df.groupby('month_num').mean()

Plot the series

In [ ]:
#plot the yearly averages
yearly_trend[metric].plot()

In [ ]:
#plot the differies yearly trend
yearly_trend['diff_series'].plot()

Now lets try a seasonal difference on the raw and differentiated series

In [ ]:
#perform seasonal diff
df['seasonal_diff'] = df[metric].diff(periods = chosen_period)
df['seasonal_diff'].plot()

Try an ACF and PACF plot

In [ ]:
#empty plot
fig, axes = plt.subplots(1, 2, figsize=(10,3))
# plot autocorrelation
acf = plot_acf(df[chosen_series].dropna(), ax=axes[0],lags=25)
# plot partial autocorrelation
pacf = plot_pacf(df[chosen_series].dropna(), ax=axes[1],lags=25)

Now let's make the model, let's start by splitting

In [ ]:
#establish the proportion
prop = chosen_proportion
#get the series
series = df[metric]
#create a length from the desired proportion
size = int(round(len(series)*prop,0))
#turn the series into a numpy array
passenger_set = series.values
#separate into train and test sets
train, test = passenger_set[0:size], passenger_set[size:len(passenger_set)]

Now for model training, look at the acf and pacf to determine the AR and MA orders, number of times you differentiated will be the order of I.

In [ ]:
#instantiate the arima model
model_ns = ARIMA(train, order=(auto_regression_order,integration_order,moving_average_order))
#fit the arima model
model_fit_ns = model_ns.fit()
#print the summary
print(model_fit_ns.summary())

Now let's evaluate

In [ ]:
#forecast the series the same length of the test set
forecasts_ns = model_fit_ns.forecast(len(test))
#compare with the testing set
mean_squared_error(forecasts_ns, test)

Now let's see how our forecast works

In [ ]:
def forecast_df(forecasts):
    #change to dataframe
    f_df = pd.DataFrame(forecasts)
    #get the proper index
    f_df['index'] = df.index[-len(test):]
    #set the index
    f_df.set_index('index',inplace=True)
    #return the df
    return f_df

Create your forecast dataframe

In [ ]:
#create the forecast df
f_ns = forecast_df(forecasts_ns)

Plot the forecasts

In [ ]:
#empty plot
fig,ax = plt.subplots()
#plot the entire series
series.plot(kind='line',ax=ax)
#plot the forecasts
f_ns.plot(kind='line',ax=ax)

Now let's take a look at our diagnostics

In [ ]:
#get the time series diagnostics plot
diag_ns = model_fit_ns.plot_diagnostics(figsize=(15, 12))
plt.show()

Let's repeat the same process with the SARIMA model

In [ ]:
#Instantiate the SARIMAX model
model_s = SARIMAX(train, order=(auto_regression_order,integration_order,moving_average_order), 
                  seasonal_order=(seasonal_ar, season_int, seasonal_ma,period))

#fit the sarima model
model_fit_s = model_s.fit()

#forecast the series the same length of the test set
forecasts_s = model_fit_s.forecast(steps=len(test))

#print the summary
model_fit_s.summary()

Evaluate

In [ ]:
#forecast using the sarima model
forecasts_s = model_fit_s.forecast(len(test))
#find the mean squared error to evaluate the model
mean_squared_error(forecasts_s, test)

Plot results

In [ ]:
#empty plot
fig,ax = plt.subplots()
#plot the whole time series
series.plot(kind='line',ax=ax)
#plot the seasonality forecasting results
f_s.plot(kind='line',ax=ax)

In [ ]:
#plot the sarima diagnostic plot
diag_s = model_fit_s.plot_diagnostics(figsize=(15, 12))
plt.show()

Now try out an auto arima model

In [ ]:
def forecast_frame(forecasts):
    #convert to the pandas dataframe
    f_df = pd.DataFrame(forecasts)
    #get the index for the forecasts
    f_df.index = df.index[-len(test):]
    #return the dataframe
    return f_df

Try toggling the seasonal on and off, change the seasonal periods in accordance to the ACF plot and seasonality analysis

In [ ]:
#fit the auto arima model
auto_arima = pm.auto_arima(train, error_action='ignore', trace=True,
                      suppress_warnings=True, maxiter=5,
                      seasonal=False,m = seasonal_period)

Evaluation

In [ ]:
#create forecasts
forecasts = auto_arima.predict(len(test))
#evaluate
mean_squared_error(forecasts, test)

Forecast and plot the results

In [ ]:
#create forecast df
forecasts_df = forecast_frame(forecasts)

In [ ]:
#empty plot
fig,ax = plt.subplots()
#plot series
series.plot(kind='line',ax=ax)
#plot forecast
forecasts_df.plot(kind='line',ax=ax)